In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

### 1. Prepare the dataset

In [ ]:
# Transformations for the input data with augmentation for the training set
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally
    transforms.RandomRotation(10),      # Randomly rotate the images by up to 10 degrees
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # Random shear and scale
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10 normalization
])

# Use the same transformation as before for the test set (without augmentation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Loading the CIFAR-10 datasets with transformations
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Classes in CIFAR-10 remain the same
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2. Define the ResNet Model

In [ ]:
# Define the BasicBlock for ResNet
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.swish = Swish()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.swish(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.swish(out)
        return out

# Define the ResNet-20 model
class ResNet20(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet20, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.swish = Swish()
        self.layer1 = self.make_layer(16, 3, stride=1)
        self.layer2 = self.make_layer(32, 3, stride=2)
        self.layer3 = self.make_layer(64, 3, stride=2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, out_channels, num_blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.swish(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = ResNet20()
net.to(device)

ResNet20(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (swish): Swish()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (swish): Swish()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (swish): Swish()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=F

### 3. Train the model

In [ ]:
# # momentum for optimizer
MOMENTUM = 0.9

#############################################
# create loss function
criterion =  nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=MOMENTUM,weight_decay=1e-4)

In [ ]:
# total number of training epochs
EPOCHS = 200
INITIAL_LR = 0.1
grad_clip = 0.1
best_val_acc = 0

# Learning rate decay
GAMMA = 0.95

# Initialize lists for later visualization
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

# Set the optimizer and the learning rate scheduler
optimizer = torch.optim.SGD(net.parameters(), lr=INITIAL_LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

print("==> Training starts!")
print("=" * 50)
for epoch in range(EPOCHS):
    net.train(True)

    # Variables to track progress
    total_examples = 0
    correct_examples = 0
    train_loss = 0

    # Training loop
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        if grad_clip is not None:
            nn.utils.clip_grad_value_(net.parameters(), grad_clip)
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_examples += targets.size(0)
        correct_examples += (predicted == targets).sum().item()
        train_loss += loss.item()

    # Store training loss and accuracy
    train_losses.append(train_loss / len(trainloader))
    train_accuracies.append(correct_examples / total_examples)

    # Validation loop
    net.eval()
    total_examples = 0
    correct_examples = 0
    val_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            _, predicted = torch.max(outputs, 1)
            total_examples += targets.size(0)
            correct_examples += (predicted == targets).sum().item()
            val_loss += loss.item()

    # Store validation loss and accuracy
    val_losses.append(val_loss / len(testloader))
    val_accuracies.append(correct_examples / total_examples)

    # Update learning rate
    scheduler.step()

    # Save the model if better
    if val_accuracies[-1] > best_val_acc:
        best_val_acc = val_accuracies[-1]
        torch.save(net.state_dict(), "resnet_91_98.ckpt")

    print(f"Epoch {epoch}: Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.4f}")

print("=" * 50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


==> Training starts!
Epoch 0: Train Loss: 0.6704, Train Acc: 0.7688, Val Loss: 0.5683, Val Acc: 0.8125
Epoch 1: Train Loss: 0.6345, Train Acc: 0.7803, Val Loss: 0.5882, Val Acc: 0.8059
Epoch 2: Train Loss: 0.6138, Train Acc: 0.7873, Val Loss: 0.5646, Val Acc: 0.8094
Epoch 3: Train Loss: 0.5765, Train Acc: 0.8021, Val Loss: 0.5321, Val Acc: 0.8226
Epoch 4: Train Loss: 0.5525, Train Acc: 0.8094, Val Loss: 0.4939, Val Acc: 0.8342
Epoch 5: Train Loss: 0.5326, Train Acc: 0.8153, Val Loss: 0.5139, Val Acc: 0.8237
Epoch 6: Train Loss: 0.5210, Train Acc: 0.8186, Val Loss: 0.4644, Val Acc: 0.8424
Epoch 7: Train Loss: 0.5031, Train Acc: 0.8253, Val Loss: 0.4711, Val Acc: 0.8412
Epoch 8: Train Loss: 0.4877, Train Acc: 0.8301, Val Loss: 0.4446, Val Acc: 0.8533
Epoch 9: Train Loss: 0.4754, Train Acc: 0.8349, Val Loss: 0.4707, Val Acc: 0.8425
Epoch 10: Train Loss: 0.4639, Train Acc: 0.8406, Val Loss: 0.4311, Val Acc: 0.8573
Epoch 11: Train Loss: 0.4520, Train Acc: 0.8420, Val Loss: 0.4546, Val Acc: 